# MCMC2.5: Dense and Sparse Matrices

This section is not necessarily related to Marcov chain Monte Carlo itself, but the use of Sparse matrices is important to speed up quantum Monte Carlo simulations.

## Dense matrix

To use dense matrices, LinearAlgebra provides wrapper to BLAS/LAPACK.

In [1]:
using LinearAlgebra

I recommend Intel MKL instead of OpenBLAS. You can check whether MKL is used by the following commands:

In [2]:
versioninfo()

Julia Version 1.0.1
Commit 0d713926f8 (2018-09-29 19:05 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin17.7.0)
  CPU: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)


In [3]:
BLAS.vendor()

:mkl